In [1]:
from pylab import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.lda import LDA
import datetime


/home/xiaohang/anaconda3/envs/xiaohang/lib/python3.5/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


In [3]:
DATA_PATH_SAT = "/home/xiaohang/Desktop/59970/project/data science new topic/SAT_Results.csv"
DATA_PATH_NYSD = "/home/xiaohang/Desktop/59970/project/data science new topic/nysd.csv"
DATA_PATH_NYS_MATH = "/home/xiaohang/Desktop/59970/project/data science new topic/NYS_Math_Test_Results_By_Grade_2006-2011_-_School_Level_-_All_Students.csv"
DATA_PATH_ATT = "/home/xiaohang/Desktop/59970/project/data science new topic/School_Attendance_and_Enrollment_Statistics_by_District__2010-11.csv"
DATA_PATH_DEMO = "/home/xiaohang/Desktop/59970/project/data science new topic/School_Demographics_and_Accountability_Snapshot_2006-2012.csv"
DATA_PATH_OUTC = "/home/xiaohang/Desktop/59970/project/data science new topic/Graduation_Outcomes_-_Classes_Of_2005-2010_-_School_Level.csv"
DATA_PATH_AP = "/home/xiaohang/Desktop/59970/project/data science new topic/AP__College_Board__2010_School_Level_Results.csv"
DATA_PATH_SIZE = "/home/xiaohang/Desktop/59970/project/data science new topic/2010-2011_Class_Size_-_School-level_detail.csv"
DATA_PATH_MAS1 = "/home/xiaohang/Desktop/59970/project/2011 data files online/masterfile11_d75_final.xlsx"
DATA_PATH_MAS2 = "/home/xiaohang/Desktop/59970/project/2011 data files online/masterfile11_gened_final.xlsx"
DATA_PATH_SUR = "/home/xiaohang/Desktop/59970/project/2011 data files online/Survey Data Dictionary.xls"
DATA_PATH_Sch = '/home/xiaohang/Desktop/59970/project/data science new topic/School_Progress_Report_2010-2011.csv'

# SAT grade

In [4]:
r_sat = pd.read_csv(DATA_PATH_SAT)
print(r_sat.dtypes)
r_sat.head()

DBN                                object
SCHOOL NAME                        object
Num of SAT Test Takers             object
SAT Critical Reading Avg. Score    object
SAT Math Avg. Score                object
SAT Writing Avg. Score             object
dtype: object


,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384


In [5]:
sat = r_sat.loc[:, {"DBN", 'SCHOOL NAME', 'SAT Critical Reading Avg. Score', 'SAT Math Avg. Score', 'SAT Writing Avg. Score'}]
sat.head()
sat.dtypes

SAT Math Avg. Score                object
SAT Critical Reading Avg. Score    object
DBN                                object
SAT Writing Avg. Score             object
SCHOOL NAME                        object
dtype: object

In [6]:
def convert_num(datframe, col_name):
    datframe[col_name] = datframe[col_name].convert_objects(convert_numeric=True)
    
convert_num(sat, 'SAT Math Avg. Score')
convert_num(sat, 'SAT Writing Avg. Score')
convert_num(sat, 'SAT Critical Reading Avg. Score')

sat['Total Avg.']= sat.sum(axis = 1)
sat.head(10)

/home/xiaohang/anaconda3/envs/xiaohang/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


,SAT Math Avg. Score,SAT Critical Reading Avg. Score,DBN,SAT Writing Avg. Score,SCHOOL NAME,Total Avg.
0,404.0,355.0,01M292,363.0,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,1122.0
1,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0
2,402.0,377.0,01M450,370.0,EAST SIDE COMMUNITY SCHOOL,1149.0
3,401.0,414.0,01M458,359.0,FORSYTH SATELLITE ACADEMY,1174.0
4,433.0,390.0,01M509,384.0,MARTA VALLE HIGH SCHOOL,1207.0
5,557.0,332.0,01M515,316.0,LOWER EAST SIDE PREPARATORY HIGH SCHOOL,1205.0
6,574.0,522.0,01M539,525.0,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",1621.0
7,418.0,417.0,01M650,411.0,CASCADES HIGH SCHOOL,1246.0
8,604.0,624.0,01M696,628.0,BARD HIGH SCHOOL EARLY COLLEGE,1856.0
9,400.0,395.0,02M047,387.0,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,1182.0


# NYSD

In [7]:
r_nysd = pd.read_csv(DATA_PATH_NYSD)
print(r_nysd.dtypes)
r_nysd.head()

the_geom       object
SchoolDist      int64
Shape_Leng    float64
Shape_Area    float64
dtype: object


,the_geom,SchoolDist,Shape_Leng,Shape_Area
0,MULTIPOLYGON (((-73.82049919995312 40.80101146...,25,175816.081000,4.436314e+08
1,MULTIPOLYGON (((-74.02552971543656 40.65147855...,20,95513.486010,2.427230e+08
2,MULTIPOLYGON (((-73.95671863064405 40.78660079...,3,52071.976405,1.134879e+08
3,MULTIPOLYGON (((-73.92133752419399 40.80085210...,4,52056.107857,5.262001e+07
4,MULTIPOLYGON (((-73.74344992332192 40.77824115...,26,125716.377003,4.249042e+08


In [ ]:
import folium


# math grade

In [8]:
r_math = pd.read_csv(DATA_PATH_NYS_MATH)
print(r_math.dtypes)
r_math.head(10)

DBN                 object
Grade               object
Year                 int64
Category            object
Number Tested        int64
Mean Scale Score    object
Level 1 #           object
Level 1 %           object
Level 2 #           object
Level 2 %           object
Level 3 #           object
Level 3 %           object
Level 4 #           object
Level 4 %           object
Level 3+4 #         object
Level 3+4 %         object
dtype: object


,DBN,Grade,Year,Category,Number Tested,Mean Scale Score,Level 1 #,Level 1 %,Level 2 #,Level 2 %,Level 3 #,Level 3 %,Level 4 #,Level 4 %,Level 3+4 #,Level 3+4 %
0,01M015,3,2006,All Students,39,667,2,5.1%,11,28.2%,20,51.3%,6,15.4%,26,66.7%
1,01M015,3,2007,All Students,31,672,2,6.5%,3,9.7%,22,71%,4,12.9%,26,83.9%
2,01M015,3,2008,All Students,37,668,0,0%,6,16.2%,29,78.4%,2,5.4%,31,83.8%
3,01M015,3,2009,All Students,33,668,0,0%,4,12.1%,28,84.8%,1,3%,29,87.9%
4,01M015,3,2010,All Students,26,677,6,23.1%,12,46.2%,6,23.1%,2,7.7%,8,30.8%
5,01M015,3,2011,All Students,28,671,10,35.7%,13,46.4%,5,17.9%,0,0%,5,17.9%
6,01M015,4,2006,All Students,49,629,20,40.8%,18,36.7%,10,20.4%,1,2%,11,22.4%
7,01M015,4,2007,All Students,40,659,4,10%,13,32.5%,18,45%,5,12.5%,23,57.5%
8,01M015,4,2008,All Students,41,655,5,12.2%,15,36.6%,18,43.9%,3,7.3%,21,51.2%
9,01M015,4,2009,All Students,39,655,4,10.3%,15,38.5%,18,46.2%,2,5.1%,20,51.3%


In [9]:
math_g = r_math.loc[:, {'DBN','Year','Mean Scale Score','Grade', 'Level 3+4 %'}]
math_g['Grade'] = math_g['Grade'].convert_objects(convert_numeric=True)
math_g = math_g.loc[(math_g['Grade']) >= 8]
math_g = math_g.loc[math_g['Year'] == 2011]
math_g.head()

/home/xiaohang/anaconda3/envs/xiaohang/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


,Level 3+4 %,Mean Scale Score,DBN,Grade,Year
111,22.9%,646,01M034,8.0,2011
280,27.9%,665,01M140,8.0,2011
346,100%,727,01M184,8.0,2011
388,26.5%,658,01M188,8.0,2011
411,18.4%,650,01M292,8.0,2011


In [10]:
len(math_g)

499

# attendance

In [11]:
r_att = pd.read_csv(DATA_PATH_ATT)
print(r_att.dtypes)
r_att.tail()

District                  object
YTD % Attendance (Avg)    object
YTD Enrollment(Avg)        int64
dtype: object


,District,YTD % Attendance (Avg),YTD Enrollment(Avg)
30,DISTRICT 31,90.98%,59373
31,DISTRICT 32,89.28%,15297
32,SPECIAL ED DISTRICT 75,83.21%,21435
33,ALTERNATIVE HIGH SCHOOLS,63.81%,7288
34,Citywide,89.99%,1002463


# percentage of diff group

In [13]:
r_demo = pd.read_csv(DATA_PATH_DEMO)
print(r_demo.dtypes)
r_demo.head()

DBN                   object
Name                  object
schoolyear             int64
fl_percent            object
frl_percent          float64
total_enrollment       int64
prek                  object
k                     object
grade1                object
grade2                object
grade3                object
grade4                object
grade5                object
grade6                object
grade7                object
grade8                object
grade9                object
grade10               object
grade11               object
grade12               object
ell_num              float64
ell_percent          float64
sped_num             float64
sped_percent         float64
ctt_num               object
selfcontained_num     object
asian_num              int64
asian_per            float64
black_num              int64
black_per            float64
hispanic_num           int64
hispanic_per         float64
white_num              int64
white_per            float64
male_num      

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158.0,56.2,123.0,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140.0,57.6,103.0,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143.0,54.8,118.0,45.2
3,01M015,P.S. 015 ROBERTO CLEMENTE,20082009,89.4,NaN,252,17,37,44,32,...,75,29.8,149,59.1,7,2.8,149.0,59.1,103.0,40.9
4,01M015,P.S. 015 ROBERTO CLEMENTE,20092010,,96.5,208,16,40,28,32,...,67,32.2,118,56.7,6,2.9,124.0,59.6,84.0,40.4


In [86]:
sub_demo = r_demo.loc[:, {'DBN', 'schoolyear','total_enrollment','asian_per','black_per','white_per','male_per','female_per'}]
sub_demo = sub_demo.loc[sub_demo['schoolyear'] == 20102011]
print(len(sub_demo))
sub_demo.head()

1498


,DBN,asian_per,black_per,male_per,female_per,schoolyear,total_enrollment,white_per
5,01M015,6.4,36.9,55.7,44.3,20102011,203,2.0
12,01M019,14.6,27.1,47.3,52.7,20102011,328,9.5
19,01M020,28.2,8.2,53.3,46.7,20102011,625,2.7
26,01M034,6.9,20.6,50.6,49.4,20102011,393,2.5
34,01M063,7.2,18.8,57.5,42.5,20102011,181,10.5


# outcome of school

In [112]:
r_outc = pd.read_csv(DATA_PATH_OUTC)
print(r_outc.dtypes)
r_outc.head()

Demographic                           object
DBN                                   object
School Name                           object
Cohort                                object
Total Cohort                           int64
Total Grads - n                       object
Total Grads - % of cohort             object
Total Regents - n                     object
Total Regents - % of cohort           object
Total Regents - % of grads            object
Advanced Regents - n                  object
Advanced Regents - % of cohort        object
Advanced Regents - % of grads         object
Regents w/o Advanced - n              object
Regents w/o Advanced - % of cohort    object
Regents w/o Advanced - % of grads     object
Local - n                             object
Local - % of cohort                   object
Local - % of grads                    object
Still Enrolled - n                    object
Still Enrolled - % of cohort          object
Dropped Out - n                       object
Dropped Ou

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,...,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
0,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2003,5,s,s,s,s,s,...,s,s,s,s,s,s,s,s,s,s
1,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2004,55,37,67.3%,17,30.9%,45.9%,...,17,30.9%,45.9%,20,36.4%,54.1%,15,27.3%,3,5.5%
2,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2005,64,43,67.2%,27,42.2%,62.8%,...,27,42.2%,62.8%,16,25%,37.200000000000003%,9,14.1%,9,14.1%
3,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78,43,55.1%,36,46.2%,83.7%,...,36,46.2%,83.7%,7,9%,16.3%,16,20.5%,11,14.1%
4,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006 Aug,78,44,56.4%,37,47.4%,84.1%,...,37,47.4%,84.1%,7,9%,15.9%,15,19.2%,11,14.1%


In [24]:
r_ap = pd.read_csv(DATA_PATH_AP)
print(r_ap.dtypes)
r_ap.head()

DBN                                     object
SchoolName                              object
AP Test Takers                          object
Total Exams Taken                       object
Number of Exams with scores 3 4 or 5    object
dtype: object


,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
0,01M448,UNIVERSITY NEIGHBORHOOD H.S.,39,49,10
1,01M450,EAST SIDE COMMUNITY HS,19,21,s
2,01M515,LOWER EASTSIDE PREP,24,26,24
3,01M539,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191
4,02M296,High School of Hospitality Management,s,s,s


In [25]:
r_ap.ix[r_ap['Number of Exams with scores 3 4 or 5'] == 's'] = np.nan
r_ap = r_ap.dropna()
r_ap.head(4)                                                                                                
                                                                                                  
                                                                                                  

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
0,01M448,UNIVERSITY NEIGHBORHOOD H.S.,39,49,10
2,01M515,LOWER EASTSIDE PREP,24,26,24
3,01M539,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191
6,02M300,"Urban Assembly School of Design and Construction,",99,117,10


In [26]:
len(r_ap)

151

# size of class

In [28]:
r_size = pd.read_csv(DATA_PATH_SIZE)
print(r_size.dtypes)
r_size.head()

CSD                                       int64
BOROUGH                                  object
SCHOOL CODE                              object
SCHOOL NAME                              object
GRADE                                    object
PROGRAM TYPE                             object
CORE SUBJECT (MS CORE and 9-12 ONLY)     object
CORE COURSE (MS CORE and 9-12 ONLY)      object
SERVICE CATEGORY(K-9* ONLY)              object
NUMBER OF STUDENTS / SEATS FILLED       float64
NUMBER OF SECTIONS                      float64
AVERAGE CLASS SIZE                      float64
SIZE OF SMALLEST CLASS                  float64
SIZE OF LARGEST CLASS                   float64
DATA SOURCE                              object
SCHOOLWIDE PUPIL-TEACHER RATIO          float64
dtype: object


,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN


In [32]:
def add_DBN_to_class(row):
    dbn = "{0:02d}{1}".format(row["CSD"], row["SCHOOL CODE"])
    return dbn
cl_size = r_size.loc[:, {'CSD','BOROUGH','SCHOOL CODE', 'AVERAGE CLASS SIZE'}]
cl_size["DBN"] = cl_size.apply(add_DBN_to_class, axis=1)
cl_size = cl_size.dropna()
cl_size.tail(10)

,SCHOOL CODE,CSD,AVERAGE CLASS SIZE,BOROUGH,DBN
27600,K564,32,23.0,K,32K564
27601,K564,32,12.0,K,32K564
27602,K564,32,26.8,K,32K564
27603,K564,32,28.5,K,32K564
27604,K564,32,24.5,K,32K564
27605,K564,32,23.7,K,32K564
27606,K564,32,25.6,K,32K564
27607,K564,32,32.5,K,32K564
27608,K564,32,26.5,K,32K564
27609,K564,32,25.0,K,32K564


In [31]:
print(len(sat), "\n", len(math_g),'\n', len(r_ap),'\n',len(cl_size))

478 
 499 
 151 
 26127


# school grade

In [33]:
r_sch_grade = pd.read_csv(DATA_PATH_Sch)
r_sch_grade.dtypes

DBN                                      object
DISTRICT                                  int64
SCHOOL                                   object
PRINCIPAL                                object
PROGRESS REPORT TYPE                     object
SCHOOL LEVEL*                            object
PEER INDEX*                             float64
2010-2011 OVERALL GRADE                  object
2010-2011 OVERALL SCORE                 float64
2010-2011 ENVIRONMENT CATEGORY SCORE    float64
2010-2011 ENVIRONMENT GRADE              object
2010-2011 PERFORMANCE CATEGORY SCORE    float64
2010-2011 PERFORMANCE GRADE              object
2010-2011 PROGRESS CATEGORY SCORE       float64
2010-2011 PROGRESS GRADE                 object
2010-2011 ADDITIONAL CREDIT             float64
2009-10 PROGRESS REPORT GRADE            object
dtype: object

In [40]:
sub_sch_grade = r_sch_grade.loc[:, {'DBN',
                                 'PRINCIPAL',
                                 'PROGRESS REPORT TYPE',
                                 'SCHOOL LEVEL*',
                                 'PEER INDEX*',
                                 '2010-2011 OVERALL GRADE',
                                 '2010-2011 ENVIRONMENT GRADE',
                                 '2010-2011 PERFORMANCE GRADE',
                                 '2010-2011 PROGRESS GRADE',
                                 '2010-2011 ADDITIONAL CREDIT'}]
sub_sch_grade.dropna()
print(len(sub_sch_grade))
sub_sch_grade.head()

1715


,2010-2011 ADDITIONAL CREDIT,DBN,PROGRESS REPORT TYPE,2010-2011 OVERALL GRADE,PEER INDEX*,2010-2011 PERFORMANCE GRADE,2010-2011 PROGRESS GRADE,SCHOOL LEVEL*,PRINCIPAL,2010-2011 ENVIRONMENT GRADE
0,0.5,01M015,EMS,C,63.61,D,C,Elementary,Irene Sanchez,B
1,0.5,01M019,EMS,B,49.48,D,A,Elementary,Jacqueline Flanagan,B
2,2.0,01M020,EMS,B,56.65,C,B,Elementary,James Lee,B
3,2.5,01M034,EMS,C,67.97,B,C,K-8,Joyce Stallings Harte,B
4,1.5,01M063,EMS,B,58.85,B,B,Elementary,Darlene Despeignes,B


In [41]:
sub_sch_grade = sub_sch_grade[sub_sch_grade['SCHOOL LEVEL*'] == 'High School']
len(sub_sch_grade)

426

# Merge

In [168]:
#result = left.join(right, lsuffix='_l', rsuffix='_r')
#concat([df1, df4], ignore_index=True)
#comb = pd.concat([sat, math_g], ignore_index=True)

comb = pd.concat([comb, r_ap], ignore_index=True)
comb = pd.concat([comb, cl_size], ignore_index=True)
#comb = comb.dropna()
comb = comb.drop_duplicates('DBN')
print(len(comb))
comb.head()


1557


,AP Test Takers,AVERAGE CLASS SIZE,BOROUGH,CSD,DBN,Grade,Level 3+4 %,Mean Scale Score,Number of Exams with scores 3 4 or 5,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SCHOOL CODE,SCHOOL NAME,SchoolName,Total Avg.,Total Exams Taken,Year
0,NaN,NaN,NaN,NaN,01M292,NaN,NaN,NaN,NaN,355.0,404.0,363.0,NaN,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,NaN,1122.0,NaN,NaN
1,NaN,NaN,NaN,NaN,01M448,NaN,NaN,NaN,NaN,383.0,423.0,366.0,NaN,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,NaN,1172.0,NaN,NaN
2,NaN,NaN,NaN,NaN,01M450,NaN,NaN,NaN,NaN,377.0,402.0,370.0,NaN,EAST SIDE COMMUNITY SCHOOL,NaN,1149.0,NaN,NaN
3,NaN,NaN,NaN,NaN,01M458,NaN,NaN,NaN,NaN,414.0,401.0,359.0,NaN,FORSYTH SATELLITE ACADEMY,NaN,1174.0,NaN,NaN
4,NaN,NaN,NaN,NaN,01M509,NaN,NaN,NaN,NaN,390.0,433.0,384.0,NaN,MARTA VALLE HIGH SCHOOL,NaN,1207.0,NaN,NaN


In [87]:
comb = pd.merge(sat, sub_sch_grade, how = 'inner', on='DBN')
comb = comb.dropna()
comb = comb.drop_duplicates()
comb = pd.merge(comb, sub_demo, how = 'inner', on = 'DBN')
comb = comb.dropna()
comb = comb.drop_duplicates()
comb = pd.merge(comb, cl_size, how = 'inner', on = 'DBN')
comb = comb.dropna()
comb = comb.drop_duplicates()
print(len(comb))
comb.head()

5907


,SAT Math Avg. Score,SAT Critical Reading Avg. Score,DBN,SAT Writing Avg. Score,SCHOOL NAME,Total Avg.,2010-2011 ADDITIONAL CREDIT,PROGRESS REPORT TYPE,2010-2011 OVERALL GRADE,PEER INDEX*,...,black_per,male_per,female_per,schoolyear,total_enrollment,white_per,SCHOOL CODE,CSD,AVERAGE CLASS SIZE,BOROUGH
0,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,2.03,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,17.7,M
1,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,2.03,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,16.6,M
2,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,2.03,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,21.3,M
3,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,2.03,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,20.3,M
4,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,2.03,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,23.7,M


In [88]:
comb.dtypes

SAT Math Avg. Score                float64
SAT Critical Reading Avg. Score    float64
DBN                                 object
SAT Writing Avg. Score             float64
SCHOOL NAME                         object
Total Avg.                         float64
2010-2011 ADDITIONAL CREDIT        float64
PROGRESS REPORT TYPE                object
2010-2011 OVERALL GRADE             object
PEER INDEX*                        float64
2010-2011 PERFORMANCE GRADE         object
2010-2011 PROGRESS GRADE            object
SCHOOL LEVEL*                       object
PRINCIPAL                           object
2010-2011 ENVIRONMENT GRADE         object
asian_per                          float64
black_per                          float64
male_per                           float64
female_per                         float64
schoolyear                           int64
total_enrollment                     int64
white_per                          float64
SCHOOL CODE                         object
CSD        

In [89]:
comb.columns.values

array(['SAT Math Avg. Score', 'SAT Critical Reading Avg. Score', 'DBN',
       'SAT Writing Avg. Score', 'SCHOOL NAME', 'Total Avg.',
       '2010-2011 ADDITIONAL CREDIT', 'PROGRESS REPORT TYPE',
       '2010-2011 OVERALL GRADE', 'PEER INDEX*',
       '2010-2011 PERFORMANCE GRADE', '2010-2011 PROGRESS GRADE',
       'SCHOOL LEVEL*', 'PRINCIPAL', '2010-2011 ENVIRONMENT GRADE',
       'asian_per', 'black_per', 'male_per', 'female_per', 'schoolyear',
       'total_enrollment', 'white_per', 'SCHOOL CODE', 'CSD',
       'AVERAGE CLASS SIZE', 'BOROUGH'], dtype=object)

# try race and sat relationship

In [90]:
comb.reset_index()

,index,SAT Math Avg. Score,SAT Critical Reading Avg. Score,DBN,SAT Writing Avg. Score,SCHOOL NAME,Total Avg.,2010-2011 ADDITIONAL CREDIT,PROGRESS REPORT TYPE,2010-2011 OVERALL GRADE,...,black_per,male_per,female_per,schoolyear,total_enrollment,white_per,SCHOOL CODE,CSD,AVERAGE CLASS SIZE,BOROUGH
0,0,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,17.7,M
1,1,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,16.6,M
2,2,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,21.3,M
3,3,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,20.3,M
4,4,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,23.7,M
5,5,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,18.0,M
6,6,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,25.0,M
7,8,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,19.0,M
8,9,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,26.4,M
9,10,423.0,383.0,01M448,366.0,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,1.0,HS,C,...,26.2,56.4,43.6,20102011,488,3.7,M448,1,27.0,M


In [99]:
data_used = comb.loc[:, {'asian_per', 'black_per','white_per', 'Total Avg.','male_per','female_per'}]
data_used = data_used.drop_duplicates()
data_used = data_used.dropna()
data_used = data_used.reset_index()
data_sample = data_used.loc[:, {'asian_per', 'black_per','white_per', 'male_per','female_per'}]
data_target = data_used.loc[:, 'Total Avg.']

In [100]:
data_sample

,asian_per,white_per,black_per,male_per,female_per
0,22.5,3.7,26.2,56.4,43.6
1,8.0,9.7,25.4,53.3,46.7
2,10.5,1.6,24.3,49.2,50.8
3,1.9,3.8,46.3,42.6,57.4
4,5.4,1.8,29.2,32.3,67.7
5,10.9,6.8,36.3,45.0,55.0
6,5.1,5.1,25.7,74.3,25.7
7,1.2,2.4,25.7,49.8,50.2
8,3.1,2.4,41.7,40.7,59.3
9,9.6,3.4,30.9,41.4,58.6


In [97]:
data_target

0      1149.0
1      1207.0
2      1194.0
3      1111.0
4      1293.0
5      1269.0
6      1051.0
7      1148.0
8      1200.0
9      1127.0
10     1176.0
11     1419.0
12     1677.0
13     1565.0
14     1419.0
15     1758.0
16     1614.0
17     1306.0
18     1233.0
19     1062.0
20     1372.0
21     1199.0
22     1227.0
23     1374.0
24     1070.0
25     1240.0
26     1239.0
27     1253.0
28     1058.0
29     1168.0
        ...  
236    1192.0
237    1345.0
238    1407.0
239    1194.0
240    1372.0
241    1196.0
242    1099.0
243    1173.0
244    1202.0
245    1287.0
246    1100.0
247    1220.0
248    1139.0
249    1151.0
250    1275.0
251    1239.0
252    1127.0
253    1470.0
254    1353.0
255    1426.0
256    1277.0
257    1279.0
258    1301.0
259    1418.0
260    1388.0
261    1235.0
262    1098.0
263    1154.0
264    1035.0
265    1060.0
Name: Total Avg., dtype: float64

In [103]:
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
knn_5 = KNeighborsClassifier(n_neighbors=3)
scores = cross_val_score(knn_5, data_sample, data_target, cv = 5)
scores

/home/xiaohang/anaconda3/envs/xiaohang/lib/python3.5/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


ValueError: Found array with 0 sample(s) (shape=(0, 5)) while a minimum of 1 is required.

In [106]:
data_used.columns.values

array(['index', 'asian_per', 'black_per', 'male_per', 'female_per',
       'Total Avg.', 'white_per'], dtype=object)

# seaborn

In [115]:
import seaborn as sns

ImportError: No module named 'seaborn'

In [113]:

sns.pairplot(data_used, x_vars= ['asian_per', 'black_per', 'male_per', 'female_per','white_per'],
             y_vars='Total Avg.', size=4, aspect=0.7, kind='reg')


ImportError: No module named 'seaborn'

,DBN,SAT Math Avg. Score,SAT Writing Avg. Score,SCHOOL NAME,SAT Critical Reading Avg. Score,Total Avg.,Level 3+4 %,Mean Scale Score,Grade,Year,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5,CSD,SCHOOL CODE,BOROUGH,AVERAGE CLASS SIZE
0,01M539,574.0,525.0,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",522.0,1621.0,99.3%,724,8.0,2011,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191,1,M539,M,25.5
1,01M539,574.0,525.0,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",522.0,1621.0,99.3%,724,8.0,2011,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191,1,M539,M,25.3
2,01M539,574.0,525.0,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",522.0,1621.0,99.3%,724,8.0,2011,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191,1,M539,M,25.0
4,01M539,574.0,525.0,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",522.0,1621.0,99.3%,724,8.0,2011,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191,1,M539,M,23.2
5,01M539,574.0,525.0,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",522.0,1621.0,99.3%,724,8.0,2011,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191,1,M539,M,27.0
9,01M539,574.0,525.0,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",522.0,1621.0,99.3%,724,8.0,2011,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191,1,M539,M,13.5
10,01M539,574.0,525.0,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",522.0,1621.0,99.3%,724,8.0,2011,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191,1,M539,M,14.9
11,01M539,574.0,525.0,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",522.0,1621.0,99.3%,724,8.0,2011,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191,1,M539,M,14.0
12,01M539,574.0,525.0,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",522.0,1621.0,99.3%,724,8.0,2011,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191,1,M539,M,14.5
13,01M539,574.0,525.0,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",522.0,1621.0,99.3%,724,8.0,2011,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191,1,M539,M,21.8


In [150]:
len(cl_size)

27611

In [182]:
sample.dtypes

Level 3+4 %                              object
Number of Exams with scores 3 4 or 5     object
AVERAGE CLASS SIZE                      float64
dtype: object

In [221]:
use_sample = comb.loc[:, {'Level 3+4 %', 'AVERAGE CLASS SIZE', 'Number of Exams with scores 3 4 or 5','Total Avg.'}]           
use_sample = use_sample.loc[use_sample['Total Avg.']!= 0]
len(use_sample)

1500

In [207]:
sample = comb.loc[:, {'Level 3+4 %', 'AVERAGE CLASS SIZE', 'Number of Exams with scores 3 4 or 5'}]
target = comb.loc[:, {'Total Avg.'}]
def turn_o(dataframe, col):
    dataframe[col] = dataframe.ix[dataframe[col] == np.nan] = 0



In [209]:
turn_o(sample, 'Level 3+4 %')
turn_o(sample, 'Number of Exams with scores 3 4 or 5')
turn_o(sample, 'AVERAGE CLASS SIZE')
sample.head()

,Level 3+4 %,Number of Exams with scores 3 4 or 5,AVERAGE CLASS SIZE
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [212]:
turn_o(target, 'Total Avg.')
target.head(20)

,Total Avg.
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [26]:
r_mas1 = pd.read_excel(DATA_PATH_MAS1)
print(r_mas1.dtypes)
r_mas1.head()

2011 School Survey for District 75 Schools\nPlease refer to the Data Dictionary file for more detailed information about the format of this data    object
Unnamed: 1                                                                                                                                          object
Unnamed: 2                                                                                                                                          object
Unnamed: 3                                                                                                                                          object
Unnamed: 4                                                                                                                                          object
Unnamed: 5                                                                                                                                          object
Unnamed: 6                                                            

,2011 School Survey for District 75 Schools Please refer to the Data Dictionary file for more detailed information about the format of this data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 1758,Unnamed: 1759,Unnamed: 1760,Unnamed: 1761,Unnamed: 1762,Unnamed: 1763,Unnamed: 1764,Unnamed: 1765,Unnamed: 1766,Unnamed: 1767
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dbn,schoolname,rr_s,rr_t,rr_p,N_s,N_t,N_p,nr_s,nr_t,...,s_q14_2,s_q14_3,s_q14_4,s_q14_5,s_q14_6,s_q14_7,s_q14_8,s_q14_9,s_q14_10,s_q14_11
2,75K004,P.S. K004,38,90,72,8,81,244,21,90,...,29,0,0,0,0,0,0,0,0,0
3,75K036,P.S. 36,70,69,44,97,43,115,139,62,...,20,27,19,9,2,6,1,2,0,0
4,75K053,P.S. K053,94,97,53,131,75,189,139,77,...,14,12,12,10,21,13,11,2,0,0


In [28]:
r_mas2 = pd.read_excel(DATA_PATH_MAS2)
print(r_mas2.dtypes)
r_mas2.head()

2011 School Survey for Community Schools\nPlease refer to the Data Dictionary file for more detailed information about the format of this data.    object
Unnamed: 1                                                                                                                                         object
Unnamed: 2                                                                                                                                         object
Unnamed: 3                                                                                                                                         object
Unnamed: 4                                                                                                                                         object
Unnamed: 5                                                                                                                                         object
Unnamed: 6                                                                  

,2011 School Survey for Community Schools Please refer to the Data Dictionary file for more detailed information about the format of this data.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 1927,Unnamed: 1928,Unnamed: 1929,Unnamed: 1930,Unnamed: 1931,Unnamed: 1932,Unnamed: 1933,Unnamed: 1934,Unnamed: 1935,Unnamed: 1936
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dbn,schoolname,rr_s,rr_t,rr_p,N_s,N_t,N_p,nr_s,nr_t,...,s_N_q14e_3,s_N_q14e_4,s_N_q14f_1,s_N_q14f_2,s_N_q14f_3,s_N_q14f_4,s_N_q14g_1,s_N_q14g_2,s_N_q14g_3,s_N_q14g_4
2,01M015,P.S. 015 Roberto Clemente,NaN,88,60,NaN,22,90,0,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01M019,P.S. 019 Asher Levy,NaN,100,60,NaN,34,161,0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01M020,P.S. 020 Anna Silver,NaN,88,73,NaN,42,367,0,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
r_sur = pd.read_excel(DATA_PATH_SUR)
print(r_sur.dtypes)
r_sur.head()

2011 NYC School Survey\nData Dictionary    object
Unnamed: 1                                 object
dtype: object


,2011 NYC School Survey Data Dictionary,Unnamed: 1
0,This data dictionary can be used with the scho...,NaN
1,NaN,NaN
2,Field Name,Field Description
3,dbn,School identification code (district borough n...
4,sch_type,"School type (Elementary, Middle, High, etc)"
